In [2]:
import os
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')

# Set path to root directory so we can access core, tasks, data, etc.
os.chdir('..')

In [3]:
import numpy
def smooth(x,window_len=11,window='hanning'):
    if x.ndim != 1:
            raise ValueError, "smooth only accepts 1 dimension arrays."
    if x.size < window_len:
            raise ValueError, "Input vector needs to be bigger than window size."
    if window_len<3:
            return x
    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
            raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"
    s=numpy.r_[2*x[0]-x[window_len-1::-1],x,2*x[-1]-x[-1:-window_len:-1]]
    if window == 'flat': #moving average
            w=numpy.ones(window_len,'d')
    else:  
            w=eval('numpy.'+window+'(window_len)')
    y=numpy.convolve(w/w.sum(),s,mode='same')
    return y[window_len:-window_len+1]

In [3]:
# Original vizsent torch
vid = pd.read_csv('data/videos/@Animated/@NewDisney/Feast/preds_old_torch/posneg.csv',
                  header=None, names=['Positive', 'Negative'])
smoothed = smooth(vid.Positive.values, window_len=48)

pd.Series(smoothed).plot()
plt.show()

In [5]:
# Sent biclass
vid = pd.read_csv('data/videos/@Animated/@NewDisney/Feast/preds/sent_biclass.csv')

# vid = pd.read_csv('data/plutchik/preds/sent_biclass_5.csv')
smoothed = smooth(vid.pos.values, window_len=48)
pd.Series(smoothed).plot()
plt.show()

In [6]:
# Plutchik
vid = pd.read_csv('data/plutchik/preds/sent_biclass_5.csv')
pd.Series(vid.pos.values).plot()
plt.show()

In [4]:
# Emo
vid = pd.read_csv('data/videos/@Animated/@NewDisney/Feast/preds/emo.csv')

w = 48
anger = smooth(vid.anger.values, window_len=w)
anticipation = smooth(vid.anticipation.values, window_len=w)
disgust = smooth(vid.anger.values, window_len=w)
fear = smooth(vid.anger.values, window_len=w)
joy = smooth(vid.joy.values, window_len=w)
sadness = smooth(vid.sadness.values, window_len=w)
surprise = smooth(vid.surprise.values, window_len=w)
trust = smooth(vid.trust.values, window_len=w)

# Trust dominates so plot without it too
df = pd.DataFrame({'anger': anger, 'anticipation': anticipation, 'disgust': disgust, 'fear': fear,
                   'joy': joy, 'sadness': sadness, 'surprise': surprise, 'trust': trust})
df.plot()
plt.show()

df = pd.DataFrame({'anger': anger, 'anticipation': anticipation, 'disgust': disgust, 'fear': fear,
                   'joy': joy, 'sadness': sadness, 'surprise': surprise})
df.plot()
plt.show()

pd.Series(trust).plot()
plt.show()